In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.functions import avg, col, count, desc
from pyspark.sql import functions as F
from pyspark.sql.window import Window
import os

# =========================================
# START SPARK SESSION
# =========================================

# Configuration
project_id = os.environ.get("PROJECT_ID", "dejadsgl")
bq_dataset = os.environ.get("BQ_DATASET", "netflix")
temp_bucket = os.environ.get("TEMP_BUCKET", "netflix-group5-temp_gl")
gcs_data_bucket = os.environ.get("GCS_DATA_BUCKET", "netflix_data_25")


# Spark configuration
sparkConf = SparkConf()
sparkConf.setMaster(os.getenv("SPARK_MASTER", "local[*]"))
sparkConf.setAppName("OutputToServingLayer")
sparkConf.set("spark.driver.memory", "2g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")

# Create the Spark session
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()

# Use the Cloud Storage bucket for temporary BigQuery export data used by the connector
spark.conf.set('temporaryGcsBucket', temp_bucket)

# Setup hadoop fs configuration for schema gs://
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

print("Spark session started.")

# =========================================
# LOAD TABLES
# =========================================

# Load data from BigQuery
df = spark.read \
            .format("bigquery") \
            .load(f"{project_id}.{bq_dataset}.unified_review_dataset")

print("\nDONE: data loaded.")

# ============================================
# BASIS: events + datumkolom op basis van review_date
# ============================================

events = (
    df
    # review_date -> event_date
    .withColumn("event_date", F.to_date("review_date"))
)

# Helper-kolommen op basis van sentiment / verificatie
events = (
    events
    .withColumn(
        "is_positive",
        F.when(F.col("sentiment") == "positive", F.lit(1)).otherwise(F.lit(0))
    )
    .withColumn(
        "is_negative",
        F.when(F.col("sentiment") == "negative", F.lit(1)).otherwise(F.lit(0))
    )
    .withColumn(
        "is_verified_flag",
        F.when(F.col("is_verified_watch") == True, F.lit(1)).otherwise(F.lit(0))
    )
)

# ============================================
# 1. fact_daily_content_metrics (partitioned by event_date)
#    Per dag x movie: review- en rating-metrics
# ============================================

fact_daily_content_metrics = (
    events
    .groupBy(
        "event_date",
        "movie_id"
    )
    .agg(
        F.count("*").alias("total_reviews"),
        F.sum("is_verified_flag").alias("verified_reviews"),
        F.avg("rating").alias("avg_rating"),
        F.avg("sentiment_score").alias("avg_sentiment_score"),
        F.sum("helpful_votes").alias("total_helpful_votes"),
        F.sum("total_votes").alias("total_votes"),
        F.avg("is_positive").alias("positive_review_ratio"),
        F.avg("is_negative").alias("negative_review_ratio")
    )
)

(
    fact_daily_content_metrics
    .write
    .format("bigquery")
    .option("table", f"{project_id}.{bq_dataset}.fact_daily_content_metrics")
    .option("partitionField", "event_date")
    .mode("overwrite")
    .save()
)

print("Written: fact_daily_content_metrics")

# ============================================
# 2. fact_user_engagement (partitioned by event_date)
#    Per dag x user: review- en rating-activiteit
# ============================================

fact_user_engagement = (
    events
    .groupBy(
        "event_date",
        "user_id"
    )
    .agg(
        F.count("*").alias("reviews_count"),
        F.countDistinct("movie_id").alias("distinct_titles_reviewed"),
        F.sum("is_verified_flag").alias("verified_reviews"),
        F.avg("rating").alias("avg_rating"),
        F.avg("sentiment_score").alias("avg_sentiment_score"),
        F.sum("helpful_votes").alias("total_helpful_votes"),
        F.avg("is_positive").alias("positive_review_ratio"),
        F.avg("is_negative").alias("negative_review_ratio"),
        F.countDistinct("device_type").alias("devices_used")
    )
)

(
    fact_user_engagement
    .write
    .format("bigquery")
    .option("table", f"{project_id}.{bq_dataset}.fact_user_engagement")
    .option("partitionField", "event_date")
    .mode("overwrite")
    .save()
)

print("Written: fact_user_engagement")

# ============================================
# 3. dim_user_segments
#    Simpele segmenten op basis van review-gedrag
# ============================================

user_agg = (
    events
    .groupBy("user_id")
    .agg(
        F.count("*").alias("total_reviews"),
        F.countDistinct("movie_id").alias("distinct_titles_reviewed"),
        F.avg("rating").alias("avg_rating"),
        F.avg("sentiment_score").alias("avg_sentiment_score"),
        F.avg("is_positive").alias("positive_review_ratio"),
    )
)

# simpele segmentlogica op basis van aantal reviews
dim_user_segments = (
    user_agg
    .withColumn(
        "user_segment",
        F.when(F.col("total_reviews") >= 50, F.lit("heavy_reviewer"))
         .when(F.col("total_reviews") >= 10, F.lit("regular_reviewer"))
         .when(F.col("total_reviews") >= 1, F.lit("occasional_reviewer"))
         .otherwise(F.lit("no_reviews"))
    )
)

(
    dim_user_segments
    .write
    .format("bigquery")
    .option("table", f"{project_id}.{bq_dataset}.dim_user_segments")
    .mode("overwrite")
    .save()
)

print("Written: dim_user_segments")

# ============================================
# 4. churn_risk_scores (heuristiek op basis van recency)
#    Hoe langer geleden de laatste review, hoe hoger churn-risico
# ============================================

# laatste review per user
last_review_per_user = (
    events
    .groupBy("user_id")
    .agg(
        F.max("event_date").alias("last_review_date")
    )
)

# globale max-datum (referentiepunt)
max_date = events.agg(F.max("event_date").alias("max_date")).collect()[0]["max_date"]

# dagen sinds laatste review + simpele score 0–1
churn_risk_scores = (
    last_review_per_user
    .withColumn(
        "days_since_last_review",
        F.datediff(F.lit(max_date), F.col("last_review_date"))
    )
    # schaal naar 0–1 (bijv. 0 dagen -> 0, >= 180 dagen -> 1)
    .withColumn(
        "churn_risk_score",
        F.when(F.col("days_since_last_review") <= 0, F.lit(0.0))
         .when(F.col("days_since_last_review") >= 180, F.lit(1.0))
         .otherwise(F.col("days_since_last_review") / F.lit(180.0))
    )
    .withColumn(
        "churn_risk_bucket",
        F.when(F.col("churn_risk_score") >= 0.8, F.lit("high"))
         .when(F.col("churn_risk_score") >= 0.4, F.lit("medium"))
         .otherwise(F.lit("low"))
    )
    .withColumnRenamed("last_review_date", "score_date")
)

(
    churn_risk_scores
    .write
    .format("bigquery")
    .option("table", f"{project_id}.{bq_dataset}.churn_risk_scores")
    .mode("overwrite")
    .save()
)

print("Written: churn_risk_scores")

# ============================================
# 5. GCS: Summary CSV reports voor business stakeholders
# ============================================

# 5a. Daily top-10 content op basis van reviews
w_top = Window.partitionBy("event_date").orderBy(F.col("total_reviews").desc())

daily_top10_content = (
    fact_daily_content_metrics
    .withColumn("rank_reviews", F.row_number().over(w_top))
    .where(F.col("rank_reviews") <= 10)
    .orderBy("event_date", "rank_reviews")
)

(
    daily_top10_content
    .coalesce(1)  # 1 CSV per run
    .write
    .mode("overwrite")
    .option("header", True)
    .csv(f"gs://{report_bucket}/daily_top10_content")
)

print("Written CSV: daily_top10_content")

# 5b. Dagelijkse engagement summary per segment
engagement_with_segment = (
    fact_user_engagement.alias("f")
    .join(
        dim_user_segments.select("user_id", "user_segment").alias("d"),
        on="user_id",
        how="left"
    )
)

engagement_summary_by_segment = (
    engagement_with_segment
    .groupBy("event_date", "user_segment")
    .agg(
        F.countDistinct("user_id").alias("unique_users"),
        F.avg("reviews_count").alias("avg_reviews_per_user"),
        F.avg("avg_rating").alias("avg_rating"),
        F.avg("avg_sentiment_score").alias("avg_sentiment_score"),
        F.avg("positive_review_ratio").alias("avg_positive_ratio")
    )
    .orderBy("event_date", "user_segment")
)

(
    engagement_summary_by_segment
    .coalesce(1)
    .write
    .mode("overwrite")
    .option("header", True)
    .csv(f"gs://{report_bucket}/engagement_summary_by_segment")
)

print("Written CSV: engagement_summary_by_segment")

# 5c. Churn summary report
churn_summary_daily = (
    churn_risk_scores
    .groupBy("score_date", "churn_risk_bucket")
    .agg(
        F.countDistinct("user_id").alias("users_in_bucket"),
        F.avg("churn_risk_score").alias("avg_churn_risk_score")
    )
    .orderBy("score_date", "churn_risk_bucket")
)

(
    churn_summary_daily
    .coalesce(1)
    .write
    .mode("overwrite")
    .option("header", True)
    .csv(f"gs://{report_bucket}/churn_summary_daily")
)

print("Written CSV: churn_summary_daily")

print("\nDONE: Output to Serving Layer.")

Spark session started.

DONE: data loaded.
Written: fact_daily_content_metrics
Written: fact_user_engagement
Written: dim_user_segments
Written: churn_risk_scores
Written CSV: daily_top10_content
Written CSV: engagement_summary_by_segment
Written CSV: churn_summary_daily


In [ ]:
spark.stop()